In [12]:
# Proper usage
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities Certificates of deposit (CDs)
Bankers' acceptances Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase) Building improvements capitalization
Machinery installation costs Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""


In [17]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN, MSO_AUTO_SIZE
from dataclasses import dataclass
from typing import List
import logging

logging.basicConfig(level=logging.INFO)

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    layer1_continued: bool = False
    layer2_continued: bool = False

class PowerPointGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.current_slide_index = 0
        self.ROWS_PER_SECTION = 28
        self.CHARS_PER_ROW = 50
        self.BULLET_CHAR = chr(0x25A0) + ' '
        self.DARK_BLUE = RGBColor(0, 50, 150)
        self.DARK_GREY = RGBColor(169, 169, 169)
        self.prev_layer1 = None
        self.prev_layer2 = None
        
    def _apply_paragraph_formatting(self, paragraph):
        """Universal paragraph formatting with version compatibility"""
        try:
            # Modern API (v0.6.18+)
            pf = paragraph.paragraph_format
            pf.alignment = PP_ALIGN.LEFT
            pf.left_indent = Inches(0.21)
            pf.first_line_indent = Inches(-0.19)
            pf.space_before = Pt(6.06)
            pf.line_spacing = 1.0
        except AttributeError:
            # Legacy API fallback
            paragraph.left_indent = Inches(0.21)
            paragraph.first_line_indent = Inches(-0.19)
            paragraph.space_before = Pt(6.06)
            paragraph.line_spacing = 1.0
            paragraph.alignment = PP_ALIGN.LEFT

    def _validate_template(self):
        """Main version validation logic"""
        required_shapes = {
            0: ["textMainBullets"],
            1: ["textMainBullets_L", "textMainBullets_R"]
        }
        for slide_idx, slide in enumerate(self.prs.slides):
            if slide_idx in required_shapes:
                missing = [name for name in required_shapes[slide_idx]
                          if not any(s.name == name for s in slide.shapes)]
                if missing:
                    raise ValueError(f"Missing shapes on slide {slide_idx+1}: {', '.join(missing)}")

    def _validate_content_placement(self, distribution):
        """Anti-duplication safeguard"""
        seen = set()
        for slide_idx, section, items in distribution:
            for item in items:
                key = (item.accounting_type, item.account_title, tuple(item.descriptions))
                if key in seen:
                    raise ValueError(f"Duplicate content detected: {key}")
                seen.add(key)


    def _calculate_chunk_size(self, items: List[FinancialItem]) -> Tuple[List[FinancialItem], FinancialItem]:
        """Calculate chunk with partial item handling"""
        lines_used = 0
        capacity = []
        
        for item in items:
            # Handle partial items from previous section
            if self.partial_item:
                item = self.partial_item
                self.partial_item = None

            # Calculate lines needed including remaining lines
            item_lines = 2  # Type + title
            if item.remaining_lines:
                item_lines += len(item.remaining_lines)
            else:
                item_lines += len(item.descriptions)
            
            if lines_used + item_lines > self.ROWS_PER_SECTION:
                # Split descriptions if possible
                if lines_used < self.ROWS_PER_SECTION:
                    available_lines = self.ROWS_PER_SECTION - lines_used - 2  # Reserve for type/title
                    if available_lines > 0:
                        partial_desc = (item.remaining_lines or item.descriptions)[:available_lines]
                        remaining_desc = (item.remaining_lines or item.descriptions)[available_lines:]
                        
                        # Create partial item for next section
                        self.partial_item = FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            [],
                            continued=True,
                            remaining_lines=remaining_desc
                        )
                        
                        # Add partial item to current chunk
                        capacity.append(FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            partial_desc,
                            item.continued
                        ))
                        lines_used += 2 + len(partial_desc)
                break
                
            lines_used += item_lines
            capacity.append(item)
            self.partial_item = None  # Reset partial after full consumption
            
        return capacity, self.partial_item

    def parse_markdown(self, md_content: str) -> List[FinancialItem]:
        """Parse markdown content into structured items"""
        items = []
        current_type = ""
        current_title = ""
        current_descs = []
        
        for line in md_content.strip().split('\n'):
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('## '):
                if current_type:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_type = line[3:].strip()
                current_title = ""
                current_descs = []
            elif line.startswith('### '):
                if current_title:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_title = line[4:].strip()
                current_descs = []
            else:
                current_descs.append(line)
                
        if current_type or current_title or current_descs:
            items.append(FinancialItem(current_type, current_title, current_descs))
            
        return items
    
    def _plan_content_distribution(self, items: List[FinancialItem]):
        """Main version distribution planning"""
        distribution = []
        content_queue = items.copy()
        slide_idx = 0
        prev_item = None

        while content_queue:
            sections = ['c'] if slide_idx == 0 else ['b', 'c']
            for section in sections:
                if not content_queue:
                    break
                section_items = []
                lines_used = 0
                while content_queue and lines_used < self.ROWS_PER_SECTION:
                    item = content_queue[0]
                    item_lines = 2 + len(item.descriptions)
                    layer1_cont = (prev_item and 
                                  item.accounting_type == prev_item.accounting_type and
                                  section != 'c')
                    modified_item = FinancialItem(
                        item.accounting_type,
                        item.account_title,
                        item.descriptions,
                        layer1_cont,
                        False
                    )
                    if lines_used + item_lines <= self.ROWS_PER_SECTION:
                        section_items.append(modified_item)
                        content_queue.pop(0)
                        lines_used += item_lines
                        prev_item = item
                    else:
                        available_lines = self.ROWS_PER_SECTION - lines_used - 2
                        if available_lines > 0:
                            split_item = FinancialItem(
                                item.accounting_type,
                                item.account_title,
                                item.descriptions[:available_lines],
                                layer1_cont,
                                False
                            )
                            section_items.append(split_item)
                            content_queue[0] = FinancialItem(
                                item.accounting_type,
                                item.account_title,
                                item.descriptions[available_lines:],
                                True,
                                False
                            )
                            break
                if section_items:
                    distribution.append((slide_idx, section, section_items))
            slide_idx += 1
        return distribution

    def _wrap_text(self, text: str) -> List[str]:
        return textwrap.wrap(text, width=self.CHARS_PER_ROW, break_long_words=True)

    def _calculate_item_lines(self, item: FinancialItem) -> int:
        """Calculate total lines required for an item"""
        wrapped_type = self._wrap_text(item.accounting_type)
        wrapped_title = self._wrap_text(item.account_title)
        wrapped_descs = [self._wrap_text(desc) for desc in item.descriptions]
        return len(wrapped_type) + len(wrapped_title) + sum(len(desc) for desc in wrapped_descs)

    def _get_section_shape(self, slide, section: str):
        """Precise shape name resolution"""
        if self.current_slide_index == 0 and section == 'c':
            target_name = "textMainBullets"
        elif self.current_slide_index > 0:
            suffix = 'L' if section == 'b' else 'R'
            target_name = f"textMainBullets_{suffix}"
        else:
            return None
        
        for shape in slide.shapes:
            if shape.name == target_name:
                return shape
        raise ValueError(f"Template missing required shape: {target_name}")


    def _apply_layer_formatting(self, paragraph, layer: int):
        """Apply formatting to layer 1 (accounting type) and layer 2 (account title)"""
        if layer == 1:
            paragraph.font.bold = True
            paragraph.font.color.rgb = RGBColor(0, 32, 96)  # Dark blue: #003296
            paragraph.paragraph_format.space_after = Pt(4)
        elif layer == 2:
            paragraph.font.italic = True
            paragraph.paragraph_format.left_indent = Inches(0.25)
            
    from pptx.oxml.xmlchemy import OxmlElement

    def _apply_bullet_formatting(self, paragraph):
        """Comprehensive XML-based bullet formatting"""
        pPr = paragraph._p.get_or_add_pPr()
        
        # Remove existing bullet elements
        for elem in pPr.xpath(".//a:buFont|.//a:buSzPct|.//a:buChar"):
            pPr.remove(elem)
        
        # Bullet size (350% of text size)
        SubElement(pPr, "a:buSzPct", val="350000")
        
        # Bullet font configuration
        SubElement(pPr, "a:buFont", 
                typeface="Calibri",
                panose="020F0502020204030204",
                pitchFamily="34",
                charset="0")
        
        # Solid square bullet character
        SubElement(pPr, "a:buChar", char="\u25A0")
        
        # Precise indentation controls
        ind = pPr.get_or_add_ind()
        ind.set('left', '302400')    # 0.3 inches
        ind.set('hanging', '201600') # 0.2 inches negative

                
    def _apply_continuation_markers(self, distribution):
        """Precise continuation tracking"""
        prev_type = prev_title = None
        for slide_idx, section, items in distribution:
            for idx, item in enumerate(items):
                item.layer1_continued = (item.accounting_type == prev_type)
                item.layer2_continued = (item.account_title == prev_title) and item.layer1_continued
                
                if idx == len(items)-1:
                    prev_type = item.accounting_type
                    prev_title = item.account_title

    def _populate_section(self, shape, items: List[FinancialItem]):
        """Modified to set bullet char grey and text black"""
        tf = shape.text_frame
        tf.clear()
        current_accounting_type = None

        for item in items:
            # Layer 1: Accounting type
            if item.accounting_type != current_accounting_type:
                p = tf.add_paragraph()
                header_text = f"{item.accounting_type} (continued)" if item.layer1_continued else item.accounting_type
                run = p.add_run()
                run.text = header_text
                run.font.size = Pt(9)
                run.font.bold = True
                run.font.color.rgb = self.DARK_BLUE
                current_accounting_type = item.accounting_type

            # Layer 2: Account title
            p = tf.add_paragraph()
            run = p.add_run()
            run.text = item.account_title
            run.font.size = Pt(9)
            run.font.italic = True

            # Layer 3: Bullet points with separate formatting
            for desc in item.descriptions:
                p = tf.add_paragraph()
                
                # Bullet character run (grey)
                bullet_run = p.add_run()
                bullet_run.text = self.BULLET_CHAR
                bullet_run.font.size = Pt(9)
                bullet_run.font.color.rgb = self.DARK_GREY  # #A9A9A9
                
                # Description text run (black)
                desc_run = p.add_run()
                desc_run.text = desc
                desc_run.font.size = Pt(9)
                desc_run.font.color.rgb = RGBColor(0, 0, 0)  # Explicit black
                
                # Apply paragraph formatting from old version
                self._apply_paragraph_formatting(p)


                
    def generate(self, md_content: str, output_path: str):
        """Main version generation logic with validation"""
        try:
            items = self.parse_markdown(md_content)
            distribution = self._plan_content_distribution(items)
            self._validate_content_placement(distribution)

            for slide_idx, section, section_items in distribution:
                if slide_idx >= len(self.prs.slides):
                    raise ValueError("Insufficient slides in template")
                slide = self.prs.slides[slide_idx]
                self.current_slide_index = slide_idx
                shape = self._get_section_shape(slide, section)
                if shape:
                    self._populate_section(shape, section_items)

            self.prs.save(output_path)
            logging.info("Successfully generated PowerPoint report")

        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            raise
    
    def _add_continuation_marks(self, items: List[FinancialItem]):
        """Mark items that continue across sections"""
        if not items:
            return
        
        # Track previous item across calls
        if self.prev_item and items:  # Add safety check
            if items[0].accounting_type == self.prev_item.accounting_type:  # FIX: Access first item
                items[0].continued = True  # FIX: Set attribute on item, not list
                if items[0].account_title == self.prev_item.account_title:  # FIX: Access first item
                    items[0].continued = True  # This line is redundant but corrected
        
        # Update previous item tracking
        self.prev_item = items[-1] if items else None


    def _advance_section(self):
        if self.current_slide_index == 0:
            # Move from Slide 0 to Slide 1 _L
            self.current_slide_index = 1
            self.current_section = 'b'
        else:
            if self.current_section == 'b':
                # Switch to _R on same slide
                self.current_section = 'c'
            else:
                # Move to next slide's _L
                self.current_slide_index += 1
                self.current_section = 'b'


# Usage
if __name__ == "__main__":
    generator = PowerPointGenerator("template.pptx")
    
    try:
        generator.generate(md_content, "financial_report.pptx")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")


INFO:root:Successfully generated PowerPoint report
